## On essaye de faire un truc qui marche ajd

In [33]:
with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        symbol = line.split(",")[0]
        # Récupère et joint le genre et l'espèce par un "_". Utile pour les bactéries.
        current_species = "_".join(line[:-1].split(",")[1].split("_")[0:2])
        species_info= {"species" : current_species, "gene_symbol" : symbol}

        
        species_info = Info(species_info)
        print(species_info)  

 

homo_sapiens
{'species': 'homo_sapiens', 'gene_symbol': 'RAD51'}
arabidopsis_thaliana
{'species': 'arabidopsis_thaliana', 'gene_symbol': 'PHYB'}
saccharomyces_cerevisiae
{'species': 'saccharomyces_cerevisiae', 'gene_symbol': 'SFA1'}
Leishmania_major
{'species': 'Leishmania_major', 'gene_symbol': 'ACP'}
escherichia_coli
{'species': 'escherichia_coli', 'gene_symbol': 'HYBE'}
mus_musculus
{'species': 'mus_musculus', 'gene_symbol': 'DMD'}
Drosophila_melanogaster
{'species': 'Drosophila_melanogaster', 'gene_symbol': 'DPP'}


## Aggrégateur

In [6]:
def Info (species_info):
    """
    Aggrège les différentes fonctions utilisées pour aller chercher les informations 
    """
    species_info = Gene(species_info)
    species_info = ProTranscript(species_info)
    return(species_info)
    

## Requête

In [4]:
from Bio import Entrez, SeqIO

def Request (species_info, db, requete, retention) :

    Entrez.email = "Chevreau.julien21@gmail.com@univ-rouen.fr"

    # Va chercher les IDs dans la base de donnée demandée
    handle = Entrez.esearch(db=db, term=requete, retmax=retention)
    records = Entrez.read(handle)
    identifiers = records["IdList"]

    # Cherche les données à partir des IDs. Résultat en format GenBank
    handle = Entrez.efetch(db=db, id=identifiers, retmax="100",
                            rettype="gbk", retmode="text")
    text = handle.read()

    # Construit un fichier temporaire pour être lu ensuite
    filename = "tmp_%s.gbk" % (db)
    with open(filename,"w") as gbk:
        gbk.write(text)

    return(filename)

species_info = {'species': 'escherichia_coli', 'gene_symbol': 'HYBE'}
requete = "%s[Orgn] AND %s[gene Name]" % (species_info["species"], species_info["gene_symbol"])

Request(species_info, "protein",requete , 5 )


'tmp_protein.gbk'

## Prot et transcrits traduits 
Aller chercher dans la base **prot** pour avoir les différents IDs des prots et transcrits associés

In [24]:
species_info = {'species': 'homo_sapiens', 'gene_symbol': 'RAD51'}

def ProTranscript (species_info):
    """
    Récupère les informations relatives aux protéines et transcrits traduits
    """

    # Requête qui récupère les entrées officielles RefSeq
    requete = "%s[Orgn] AND %s[gene Name] AND srcdb refseq [properties]" % (species_info["species"], species_info["gene_symbol"])

    # Passage de la requête à la fonction qui va chercher les informations
    prot_file = Request(species_info, "protein",requete ,60)

    # Parcours du fichier temporaire créé pour en extraire les informations
    for seq_record in SeqIO.parse(prot_file, "genbank"):
        try : # Si db_source n'existe pas pour le record, passe au suivant
            db_source = seq_record.annotations["db_source"].split(" ")

            if db_source[0] == "REFSEQ:" :
                if "transcript_id" not in species_info :
                    species_info["transcript_id"] = [db_source[-1]]
                    species_info["prot_id"] = [seq_record.id]
                else :
                    species_info["transcript_id"].append(db_source[-1])
                    species_info["prot_id"].append(seq_record.id)
        except :
            continue

    # Indique si aucun transcrit/protéine n'est trouvé            
    if "transcript_id" not in species_info :
                species_info["transcript_id"] = ["No data found"]
                species_info["prot_id"] = ["No data found"]
    return(species_info)

ProTranscript(species_info)

{'species': 'homo_sapiens',
 'gene_symbol': 'RAD51',
 'transcript_id': ['NM_001164270.2',
  'NM_001164269.2',
  'NM_133487.4',
  'NM_002875.5',
  'XM_054378571.1',
  'XM_054378570.1',
  'XM_054378569.1',
  'XM_054378568.1',
  'XM_054378567.1',
  'XM_054378566.1',
  'XM_047432925.1',
  'XM_011521861.3',
  'XM_011521857.3',
  'XM_011521860.3',
  'XM_011521859.3',
  'XM_011521858.3'],
 'prot_id': ['NP_001157742.1',
  'NP_001157741.1',
  'NP_597994.3',
  'NP_002866.2',
  'XP_054234546.1',
  'XP_054234545.1',
  'XP_054234544.1',
  'XP_054234543.1',
  'XP_054234542.1',
  'XP_054234541.1',
  'XP_047288881.1',
  'XP_011520163.2',
  'XP_011520159.2',
  'XP_011520162.1',
  'XP_011520161.1',
  'XP_011520160.1']}

## Transcrits non traduits
Aller dans **nucleotide** pour les transcrits non traduits

In [ ]:
species_info = {'species': 'homo_sapiens', 'gene_symbol': 'RAD51', 'transcript_id': ['NM_001164270.2', 'NM_001164269.2', 'NM_133487.4', 'NM_002875.5', 'XM_054378571.1', 'XM_054378570.1', 'XM_054378569.1', 'XM_054378568.1', 'XM_054378567.1', 'XM_054378566.1', 'XM_047432925.1', 'XM_011521861.3', 'XM_011521857.3', 'XM_011521860.3', 'XM_011521859.3', 'XM_011521858.3'], 'prot_id': ['NP_001157742.1', 'NP_001157741.1', 'NP_597994.3', 'NP_002866.2', 'XP_054234546.1', 'XP_054234545.1', 'XP_054234544.1', 'XP_054234543.1', 'XP_054234542.1', 'XP_054234541.1', 'XP_047288881.1', 'XP_011520163.2', 'XP_011520159.2', 'XP_011520162.1', 'XP_011520161.1', 'XP_011520160.1']}

def NCrna(species_info) :

    transcript_file = Request(species_info,"nucleotide",50)

    for seq_record in SeqIO.parse(transcript_file, "genbank"):
        print(seq_record.annotations["molecule_type"])
        if seq_record.annotations["molecule_type"] == "mrna" :
            if not seq_record.id in species_info['transcript_id']:
                species_info['transcript_id'].append(seq_record.id + " (Not translated)")
    
    return(species_info)

NCrna(species_info)

## Gène
Aller dans **gene** pour l'official full name et le gene access number

In [15]:
import re
species_info = {'species': 'escherichia_coli', 'gene_symbol': 'HYBE'}
def Gene (species_info) :

    # Requête pour la base de donnée
    requete = "%s[Orgn] AND %s[gene Name] AND Refseq [Keyword]" % (species_info["species"], species_info["gene_symbol"])
    gene_file = Request(species_info,"gene", requete, 1)

    # Ouverture et lecture du fichier temporaire contenant les informations du gène
    with open(gene_file,"r") as file:
        content = "".join(file.readlines())

        # Expressions régulières pour trouver ID et nom officiel
        pattern_id = re.search(r"ID:\s(\d*)",content)
        pattern_official = re.search(r"Name:\s(.*)\s\[",content)
        pattern_official_alt = re.search(r"(.*)\s\[.*\]",content)

        # Population du dictionnaire en cas de match
        if pattern_official != None :
            species_info["official_name"] = pattern_official.group(1)
        elif pattern_official_alt != None : # Pattern alternatif dans le cas où le premier n'est pas trouvé
            species_info["official_name"] = pattern_official_alt.group(1)

        if pattern_id != None :
            species_info["gene_id"] = pattern_id.group(1)
    return(species_info)

Gene(species_info)


{'species': 'escherichia_coli',
 'gene_symbol': 'HYBE',
 'official_name': 'hydrogenase 2-specific chaperone',
 'gene_id': '947483'}

## Section Test

In [ ]:

        # # Lit toute les lignes et ne garde que la première entrée
        # content = "".join(file.readlines())
        # entry_one = re.search(r"1.(.*)\n\n", content, flags=re.S).group(1)

# ((Hybe[Gene Name]) AND Escherichia coli[Organism]) AND Refseq[Keyword]


import re
with open ("tmp_gene.gbk", "r") as file :
    contenu = "".join(file.readlines())
    entry_one = re.search(r"\n(.*)\n\n2\.", contenu, flags=re.S)
    
    print (contenu)

## Conversion gbk en fasta

In [ ]:
from Bio import SeqIO
SeqIO.convert("all_from_eu.gbk",'genbank',"all_from_eu.fasta","fasta")

In [ ]:
from Bio import SeqIO
gb_file = "f1.gbk"
features = {}
genes = {}
for seq_record in SeqIO.parse(gb_file, "genbank"):
    for feat in seq_record.features :
        if feat.type in features :
            features[feat.type]+=1
        else : 
            features[feat.type] = 1
        if "gene" in feat.qualifiers:
            for gene in feat.qualifiers["gene"]:
                genes[gene] = 1
                
print(features)
print(genes)